#### Isolate Decison By Bins

In [20]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os 
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Active/Swing Decisons/')

In [21]:
stat = pd.read_csv('./data/swing_data/asv_swing%.csv').drop(columns=['Unnamed: 0', 'Contact%'])

In [22]:
# Create bins and labels
n_per_bin = len(stat) // 3
swing_bins = pd.qcut(stat['Swing%'], q=3, labels=['Bin '+str(i+1) for i in range(3)])
stat['swing_bin_number'] = swing_bins.cat.codes + 1

# Add the bins to the dataframe and calculate mean values for each bin
swing_analysis = stat.assign(swing_bin=swing_bins).groupby('swing_bin').agg({
    'Swing%': 'mean',
    'chase_shadow_value': 'mean',
    'swing_value': 'mean',
    'PA': 'count'
}).round(4)

print("Number of players in each bin:", n_per_bin)
print("\nBin Analysis:")
print(swing_analysis)

Number of players in each bin: 774

Bin Analysis:
           Swing%  chase_shadow_value  swing_value   PA
swing_bin                                              
Bin 1      0.4225            106.4228     118.4447  776
Bin 2      0.4779            100.2057     100.9096  781
Bin 3      0.5375             93.1184      80.0989  766


C:\Users\dalto\AppData\Local\Temp\ipykernel_25404\3747203935.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  swing_analysis = stat.assign(swing_bin=swing_bins).groupby('swing_bin').agg({


In [23]:
# Create a new dataframe with normalized values
normalized_stats = pd.DataFrame()
normalized_stats['batter'] = stat['batter']
normalized_stats['game_date'] = stat['game_date']
normalized_stats['PA'] = stat['PA']
# For each metric, calculate normalized values based on bin means
for metric in ['Swing%', 'chase_shadow_value', 'swing_value']:
    # Get the bin means for the current metric
    bin_means = swing_analysis[metric]
    
    # Calculate normalized values (100 + percentage difference from bin mean * 100)
    normalized_stats[f'normalized_{metric}'] = stat.apply(
        lambda row: 100 * (1 + (row[metric] - bin_means[row['swing_bin_number'] - 1]) / bin_means[row['swing_bin_number'] - 1]),
        axis=1
    )

C:\Users\dalto\AppData\Local\Temp\ipykernel_25404\1482308486.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda row: 100 * (1 + (row[metric] - bin_means[row['swing_bin_number'] - 1]) / bin_means[row['swing_bin_number'] - 1]),
C:\Users\dalto\AppData\Local\Temp\ipykernel_25404\1482308486.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda row: 100 * (1 + (row[metric] - bin_means[row['swing_bin_number'] - 1]) / bin_means[row['swing_bin_number'] - 1]),
C:\Users\dalto\AppData\Local\Temp\ipykernel_25404\1482308486.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futur

In [24]:
stat['normalized_chase'] = normalized_stats['normalized_chase_shadow_value']
stat = stat.drop(columns=['swing_bin_number', 'Swing%'])
stat = stat[stat['PA'] > 50]

In [18]:
stat.to_csv('./data/stat_values/swing_bin.csv')